# TD 3: Equation de Schrödinger et puits de potentiel

Ce notebook vise à illustrer le TD3 dans lequel a été résolu l'équation de Schrödinger dans le cas des états stationnaires de la fonction d'onde. On cherche ici à proposer une solution à l'équation de Schrödinger pour un paquet d'onde gaussien. La visualisation proposera également une visualisation en 2D.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle

### Implémentation de fonctions utiles

L'intégration de l'équation de Schrödinger peut se faire en utilisant un schéma de Crank-Nicolson:

$$
-\frac{\hbar^2}{2m}\Delta \psi + V(x)\psi(x,t) = i\hbar \frac{\partial \psi}{\partial t}
$$



In [2]:
def psi0(x, y, x0, y0, sigma=0.5, k=15*np.pi):
    """
    Wave function for the initial time step (t=0)
    Initial position (x0, y0)
    Default parameters:
        - sigma = 0.5 -> Gaussian dispersion
        - k = 15*np.pi -> Wave number, proportional to the momentum
    
    """
    return np.exp(-((x-x0)**2 + (y-y0)**2)/(4*sigma**2)) * np.exp(1j*k*x)

In [ ]:
# =============================================================================
# Parameters
# =============================================================================

L = 8 # Well of width L. Shafts from 0 to +L.
Dy = 0.05 # Spatial step size.
Dt = Dy**2/4 # Temporal step size.
Nx = int(L/Dy) + 1 # Number of points on the x axis.
Ny = int(L/Dy) + 1 # Number of points on the y axis.
Nt = 500 # Number of time steps.
rx = -Dt/(2j*Dy**2) # Constant to simplify expressions.
ry = -Dt/(2j*Dy**2) # Constant to simplify expressions.

# Initial position of the center of the Gaussian wave function.
x0 = L/5
y0 = L/2

# Parameters of the double slit.
w = 0.6 # Width of the walls of the double slit.
s = 0.8 # Separation between the edges of the slits.
a = 0.2 # Aperture of the slits.

# Indices that parameterize the double slit in the space of points.
# Horizontal axis.
j0 = int(1/(2*Dy)*(L-w)) # Left edge.
j1 = int(1/(2*Dy)*(L+w)) # Right edge.
# Eje vertical.
i0 = int(1/(2*Dy)*(L+s) + a/Dy) # Lower edge of the lower slit.
i1 = int(1/(2*Dy)*(L+s))        # Upper edge of the lower slit.
i2 = int(1/(2*Dy)*(L-s))        # Lower edge of the upper slit.
i3 = int(1/(2*Dy)*(L-s) - a/Dy) # Upper edge of the upper slit.

# We generate the potential related to the double slit.
v0 = 200
v = np.zeros((Ny,Ny), complex) 
v[0:i3, j0:j1] = v0
v[i2:i1,j0:j1] = v0
v[i0:,  j0:j1] = v0
    
Ni = (Nx-2)*(Ny-2)  # Number of unknown factors v[i,j], i = 1,...,Nx-2, j = 1,...,Ny-2
